In [1]:
from glovar import BRAND_LIST,IPHONE_LIST,STORAGE_LIST
import re

def get_phone_attribute_spain(info_txt):
    """
    infomation txt: e.g "apple iphone 12 mini púrpura, 256gb, 5g, 5.4 oled super retina xdr, chip a14 bionic, ios"
    """
    brand, phone_type, storage_and_memory, color = None,None,None,None
    ram = None
    is_5g = False

    # info = info_txt.strip().split(",")

    # get storage_and_memory info
    for storage in STORAGE_LIST:
        if storage in info_txt:
            storage_and_memory = storage
    
    # get brand and phone_type info 
    start,end = 0,0
    if "iphone" in info_txt or "apple" in info_txt:
        brand = "Apple"
        start = info_txt.strip().split(",")[0].split().index("apple")
        phone_type = " ".join(info_txt.strip().split(",")[0].split()[start+1:])
    elif "samsung" in info_txt:
        brand = "Samsung"
        start = info_txt.strip().split(",")[0].split().index("samsung")
        phone_type = " ".join(info_txt.strip().split(",")[0].split()[start+1:])

    elif "xiaomi" in info_txt:
        brand = "Xiaomi"
        start = info_txt.strip().split(",")[0].split().index("xiaomi")
        phone_type = " ".join(info_txt.strip().split(",")[0].split()[start+1:])
    elif "oppo" in info_txt:
        brand = "OPPO"
        start = info_txt.strip().split(",")[0].split().index("oppo")
        phone_type = " ".join(info_txt.strip().split(",")[0].split()[start+1:])

    if phone_type and "(" in phone_type:
        phone_type = phone_type.split("(")[0]
    
    if "ram" in info_txt:
        info = info_txt.strip().split(",")
        for it in info:
            if "ram" in it:
                ram = it.split()[0]
    
    if info_txt.count("gb") >= 2 and "ram" not in info_txt:
        index1 = info_txt.find("gb")
        index2 = info_txt.find("gb",index1+1)
        ram = info_txt[index2 - 1:index2 + 2]

    # get is_5g info
    if "5g" in info_txt:
        is_5g = True

    # get color info
    color = info_txt.strip().split(",")[1]

    return brand, phone_type, storage_and_memory, ram, is_5g, color

def extract_and_save_spain(book,sheet,file_name):
    sheet["C1"], sheet["D1"], sheet["E1"], sheet["F1"],sheet["G1"],sheet["H1"] = "brand", "phone_type", "storage_and_memory", "ram", "is_5g", "color"
    sheet["I1"] = "price"
    for i,row in enumerate(sheet.values):
        source_txt, price = row[0],row[1]
        if i > 0 :
            if source_txt and price:
                if "," not in source_txt:
                    continue
                txt = source_txt.lower()
                txt = txt.strip().replace(" gb","gb")
                txt = txt.strip().replace(" tb","tb")
                price = float(re.findall(r"\d+\.?\d*",price)[0])
                # print(txt, price)

                brand, phone_name, storage_and_memory, ram, is_5g, color = get_phone_attribute_spain(txt)
                # print(brand, phone_type, storage_and_memory, is_5g, color)
                brand_place = "C" + str(i+1)
                phone_type_place = "D" + str(i+1)
                storage_and_memory_place = "E" + str(i+1)
                ram_place = "F" + str(i+1)
                is_5g_palce = "G" + str(i+1)
                color_place = "H" + str(i+1)
                price_place = "I" + str(i+1)

                sheet[brand_place] = brand
                sheet[phone_type_place] = phone_name
                sheet[storage_and_memory_place] = storage_and_memory
                sheet[ram_place] = ram
                sheet[is_5g_palce] = is_5g
                sheet[color_place] = color
                sheet[price_place] = price
            else:
                print("spain badcase in line {}!!!".format(i))

    book.save(file_name)

def create_new_sheet_spain(book,new_sheet,file_name):
    if "spain_test" in book.sheetnames:
        book.remove(book["spain_test"])
    ws = book.create_sheet('spain_test')
    ws["A1"],ws["B1"],ws["C1"],ws["D1"],ws["E1"] = "BRAND", "TYPE", "RAM_AND_STORAGE", "TIME", "PRICE" 
    ws["F1"],ws["G1"],ws["H1"],ws["I1"],ws["J1"],ws["K1"],ws["L1"] = "GERMAN","FRANCE","UK","ITALY","SPAIN","FINLAND","CZECH"


    spain_phone = set()
    j = 2
    for i,row in enumerate(new_sheet.values):
        if i > 0:
            brand, phone_type, storage, ram, is_5g, color, price = row[2],row[3],row[4],row[5],bool(row[6]),row[7],row[8]
            if brand and phone_type and storage and price:
                if is_5g and "5g" not in phone_type:
                    phone_type = phone_type + " 5g "
                phone_type = " ".join(phone_type.split())
                
                unique_phone_id  = " ".join([brand, phone_type, storage, str(price)])
                # print(unique_phone_id)

                # unique_phone_id = " "brand + " " + phone_type + " " + storage + " " + price + " "

                # if unique_phone_id not in spain_phone:
                brand_place = "A" + str(j)
                phone_type_place = "B" + str(j)
                storage_and_memory_place = "C" + str(j)
                price_place = "J" + str(j)
                
                ws[brand_place] = brand
                ws[phone_type_place] = phone_type
                ws[storage_and_memory_place] = storage
                ws[price_place] = price
                j += 1
            
                # spain_phone.add(unique_phone_id)

    # for t in sorted(list(spain_phone)):
    #     print(t)

    book.save(file_name)